In [18]:
from scripts.software_frs import frs_traceability2
FRS = frs_traceability2
from officelib.xllib import *
from officelib.const import xlconst as xlc
import sys
import os
import re

from datetime import datetime
import importlib
import jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter2 as iie
from scripts.tools.redmine_cache import RedmineClient

In [19]:
_cache = None
_age = None
_client = None

def _get_client():
    global _client
    if _client is None:
        _client = RedmineClient()
    return _client
def _download_issues():
    _client = _get_client()
    return _client.get_filtered([('project.identifier', '==', 'pbssoftware')])

def get_issues():
#     global _cache, _age
#     if not _cache or (datetime.now() - _age).total_seconds() > (8*3600):  # 8 hr
#         _cache = _download_issues()
#         _age = datetime.now()
#     return _cache
    return _download_issues()

In [20]:
def filter_relevant_issues(issues, rel_cb):
    relevant = {}
    for v in issues.values():
        if rel_cb(v):
                relevant[v.id] = v
    return relevant

def load_frs_from_issuetracker(rel_cb):
    issues = get_issues()
    relevant = filter_relevant_issues(issues, rel_cb)
    
    reqs = iie.IssuetrackerParser(["URS", "FRS", "SDS"]).parse_all(relevant.values())
    return reqs, relevant


In [21]:
def paste_data(ws, data):
    cells = ws.Cells
    cr = cells.Range
    
    di = 2
    hi = 3
    
    header_start = cr("A1")
    
    frs_start = cr("A2")
    frs_end = header_start.Offset(len(data), 1)

    id_start = frs_start.Offset(1, 3)
    id_end = frs_end.Offset(1, 3)
    

    paste_start = header_start
    paste_end = frs_end.Offset(1, len(data[0]))

    xl = ws.Application
    
    with screen_lock(xl):
        print("Pasting test data")
        paste_range = cr(paste_start, paste_end)
        paste_range.Clear()
        cr(id_start, id_end).NumberFormat = "@"
        cr(paste_range.Cells(1, 5), paste_range.Cells(len(data), 5)).NumberFormat = "@"
        paste_range.Value2 = data         

        # This has to come before IndentLevel is set,
        # or it gets fucked up for some reason even
        # though it seems to work correctly when performing
        # the operation manually
        print("Applying alignment formatting")
        col = paste_range.Columns(2)
        col.ColumnWidth = 100
        col.VerticalAlignment = xlc.xlTop
        paste_range.Columns(2).WrapText = True
        
        # Vertical alignment should be top for all
        for i in range(1, len(data[0])+1):
            paste_range.Columns(i).VerticalAlignment = xlc.xlTop
            paste_range.Columns(i).HorizontalAlignment = xlc.xlLeft
        
        # Create a new range to iterate over
        # this improves performance by minimizing the number of 
        # calls that have to be across the COM server to apply
        # the indents
    
        print("Applying row formatting")
        indent_range = cr(paste_start, paste_start.Offset(len(data), 2))
        for i, (d, row) in enumerate(zip(data, indent_range.Rows), 1):
            count = d[0].count(".") * 2
            row.IndentLevel = count
            if count == 0:
                rint = paste_range.Rows(i).Interior
                rint.Pattern = xlc.xlSolid
                rint.PatternColorIndex = xlc.xlAutomatic
                rint.ThemeColor = xlc.xlThemeColorAccent6
                rint.TintAndShade = 0.6
                
        # Document header row formatting
        rint = paste_range.Rows(1).Interior
        rint.Pattern = xlc.xlSolid
        rint.PatternColorIndex = xlc.xlAutomatic
        rint.ThemeColor = xlc.xlThemeColorDark1
        rint.TintAndShade = -0.249977111117893
        
        print("Applying column autofit")
        # fit after filter to account for width of filter icon
        for i in (1, 3, 4, 5):
            paste_range.Columns(i).AutoFit()

        print("Applying row autofit")
        paste_range.Rows.AutoFit()

In [38]:
from officelib.xllib import *
import re

# unlike the Reference class's item key, the key for the Node class
# is (TYPE+FIRSTNUM, OTHERNUMS). e.g. ('URS123', '4.5') instead of
# ('URS', '123.4.5')
_imkey = re.compile(r"(%s)(\d+)\.?([\d\.]*)" % "|".join(['URS', 'FRS', 'SDS'])).match

def key_match(key):
    m = _imkey(key)
    if m:
        type, first, others = m.groups()
        return type+first, others
    return key, ""

def build_frs_tree(all_items, type):
    root = FRS.Root(key_match)
    for frs, req in all_items.items():
        if req.type != type:
            continue
        node = root.add(frs, int(req.obs))
        node.text = req.text
        node.refs = req.refs
        node.milestone = req.milestone
        node.priority = req.priority
    return root

def make_paste_data2(reqs, typ, include_obs=False):
    data = [["%s Number"%typ, "Text", "References", "Criticality", "Release Version", "Sort Order"]]
    seen = {}
    thekey = lambda req: [int(x or 0) for x in req.num.split(".")]
    reqs = [r for r in reqs if r.type == typ and (include_obs or not r.obs)]
    for req in sorted(reqs, key=thekey):
        rkey = req.type + req.num.split(".")[0]
        if rkey in seen:
            so = "=R[-1]C + 1"
        else:
            seen[rkey] = so = (len(seen) + 1) * 1000
        data.append((req.tag, numberify(req.text), "\n".join(req.refs), req.priority, req.milestone, so))
    return data

import re
_numberify_sub = re.compile(r"^(#+|\*+)", flags=re.MULTILINE).sub

def numberify(text):
    
    stack = None
    last = None

    def _numberify(m):
        nonlocal last, stack
        s = m.group(1)
        if s[0] == "*":
            return "  " * len(s) + "*"
        if last is None:
            stack = [1]
            last = s
            lastn = 1
            return "  1)"
        else:
            if len(s) < len(last):
                stack.pop()
            elif len(s) > len(last):
                stack.append(0)
            last = s
            stack[-1] += 1
            return "  " + ".".join(map(str, stack)) + ")"
    return _numberify_sub(_numberify, text)

def _sanitycheck(reqs):
    seen = {}
    dup = []
    for r in reqs:
        if r.tag and not r.obs:
            if r.tag in seen and not seen[r.tag][0].equals(r):
                dup.append(r)
                r2, b = seen[r.tag]
                if b == False:
                    dup.append(r2)
                    seen[r.tag] = [r2, True]
            else:
                seen[r.tag] = [r, False]
    if dup:
        s = sorted(dup, key=lambda r: r.tag)
        strings = "\n".join("%s: (#%d) %s" % (r.tag, r.source.id, r.text) for r in s)
        raise ValueError("Found duplicate items:\n%s"%(strings))
        
def _sanitycheck2(reqs):
    seen = {}
    dup = []
    for r in reqs:
        if r.tag:
            if r.tag in seen and not seen[r.tag][0].equals(r):
                dup.append(r)
                r2, b = seen[r.tag]
                if b == False:
                    dup.append(r2)
                    seen[r.tag] = [r2, True]
            else:
                seen[r.tag] = [r, False]
    if dup:
        s = sorted(dup, key=lambda r: r.tag)
        strings = "\n".join("%s: (#%d) %s" % (r.tag, r.source.id, r.text) for r in s)
        print("Found duplicate items:\n%s"%(strings))

def main(type='URS', rel_cb=lambda x: True):

    print("Downloading issues")
    issues = get_issues()
    print("Parsing relevant issues")
    relevant = filter_relevant_issues(issues, rel_cb)
    reqs = iie.IssuetrackerParser(["URS", "FRS", "SWDS"]).parse_all(relevant.values())
    _sanitycheck(reqs)
    xl = Excel()
    with screen_lock(xl):
        print("Compiling data for final matrix")
        data = make_paste_data2(reqs, type)  
        ws = FRS.get_matrix_sheet(xl)
        paste_data(ws, data)
        print("Done")
        
def combo_main(types, rel_cb=lambda x:True, include_obs=False):
    print("Downloading issues")
    issues = get_issues()
    print("Parsing relevant issues")
    relevant = filter_relevant_issues(issues, rel_cb)
    reqs = iie.IssuetrackerParser(["URS", "FRS", "SWDS"]).parse_all(relevant.values())
    _sanitycheck2(reqs)
    xl = Excel()
    with HiddenXl(xl):
        with screen_lock(xl):
            for typ in types:
                print("Compiling data for final matrix '%s'"%typ)
                data = make_paste_data2(reqs, typ, include_obs)  
                ws = FRS.get_matrix_sheet(xl)
                paste_data(ws, data)
                print("Done")

In [40]:
iie = importlib.reload(iie)
_ignore = {3194, 3287}
_sprints = [
    'Legacy',
    '3.0',
    '3.0.1',
    '3.0.2',
    '3.0.3',
    '3.0.4',
    '3.0.5',
    '3.0.6',
    '3.0.7',
    '3.0.8'
]
_ssprints = set(_sprints)

def relevant(i):
    return i.sprint_milestone.name in _ssprints and i.id not in _ignore and i.status != "Rejected"

trace_path = 'C:\\Users\\Nathan\\Documents\\Dropbox\\FRS'
user_tests = 'FRS Tests 181127.xlsx'
p1 = os.path.join(trace_path, user_tests)
# main('URS', relevant)
# main('FRS', relevant)
combo_main(['URS', 'FRS'], relevant)

Parsing relevant issues
Found duplicate items:
FRS3652.5: (#3652) Broken Sensor Mode Triggers 
FRS3652.5: (#3652) Broken Sensor mode: when the user has requested Auto mode but the DO PV reported by the sensor is either above the DO "Valid High (DO%)" setting or below the DO "Valid Low (DO%)" setting, the software outputs the average of the last 100 seconds of O2 and N2 output.
FRS4090.12.1: (#4090) Alarms may be configured to be ignored, as if they were never triggered. 
FRS4090.12.1: (#3224) An alarm may be configured as "notify", causing the alarm to be generated if the trigger conditions are met. Otherwise, no alarm will be generated. 
Compiling data for final matrix 'URS'
Pasting test data
Applying alignment formatting
Applying row formatting
Applying column autofit
Applying row autofit
Done
Compiling data for final matrix 'FRS'
Pasting test data
Applying alignment formatting
Applying row formatting
Applying column autofit
Applying row autofit
Done


In [ ]:
xl = Excel()
while True:
    try:
        wb = xl.Workbooks(1)
        wb.Close(False)
    except:
        break
xl.Quit()
del xl

In [ ]:
def rowify():
    try:
        xl
    except NameError:
        xl = Excel()
    rows = xl.Selection.Value2
    out = []
    for tag, text in rows:
        tag = "*" * tag.count(".") + " " + "*" + tag + "*"
        
        out.append(tag + " " + text)
    return "\n".join(out)

In [10]:
iss = get_issues()[989]

In [12]:
import json, clipboard
print(json.dumps(json.loads(clipboard.paste()), indent=4))

{
    "issue": {
        "done_ratio": 0,
        "id": 989,
        "spent_hours": 0.0,
        "total_estimated_hours": 6.0,
        "start_date": "2016-05-23",
        "total_spent_hours": 0.0,
        "status": {
            "name": "Unit Test",
            "id": 10
        },
        "author": {
            "name": "Daniel Giroux",
            "id": 4
        },
        "created_on": "2011-12-20T23:53:52Z",
        "tracker": {
            "name": "Specification",
            "id": 13
        },
        "assigned_to": {
            "name": "James Small",
            "id": 42
        },
        "fixed_version": {
            "name": "3.0",
            "id": 52
        },
        "subject": "Login Behavior",
        "priority": {
            "name": "Normal",
            "id": 4
        },
        "attachments": [
            {
                "thumbnail_url": "https://issue.pbsbiotech.com/attachments/thumbnail/389",
                "filename": "valid_user_name.png",
               